In [ ]:
!pip install sovai[full]

## SEC 10-K filings

You can run the following commands to retrieve data (`df`) using `sov.data`:

To fetch the **latest data** for a specific query:

```python
df = sov.data("query")
```

To fetch the **full historical data** for a specific query:

```python
df = sov.data("query", full_history=True)
```

To fetch the **full data** multiple **tickers** or identifiers like **cusip** and **openfigi**:

```python
df = sov.data("query", tickers=["9033434", "IB94343", "43432", "AAPL"])
```

To filter **any dataframe** just write some queries:

```python
df.filter(["cash_short_term > 10m","start with ticker A","negative profits" ])
```


In [15]:
import sovai as sov
import pandas as pd

sov.token_auth(token="visit https://sov.ai/profile for your token")

data = sov.data("sec/10k", tickers=["AAPL"], limit=1)

In [16]:
data.to_parquet("park.parquet")

You don't have to downlaod the entire dataset, instead look at the companies that filed that week, and simply do an analysis of them. Future edits would include sentiment analysis.

In [12]:
import pandas as pd

# Define mandatory sections
mandatory_sections = [
    "FINANCIAL_STATEMENTS",
    "RISK_FACTORS",
    "MANAGEMENT_DISCUSSION",
    "MANAGEMENT",
    "COMPENSATION",
    "EXHIBITS",
    "BUSINESS",
    "CONTROLS_AND_PROCEDURES",
    "PRINCIPAL_STOCKHOLDERS",
    "ACCOUNTING_FEES"
]

# 1. Filter for mandatory sections using isin (vectorized operation)
mandatory_data = data[data['section'].isin(mandatory_sections)].copy()

# 2. Extract year from the 'date' column
mandatory_data['year'] = mandatory_data['date'].dt.year

# 3. Calculate word count using vectorized string operations
mandatory_data['word_count'] = mandatory_data['full_text'].str.count(r'\w+')

# 4. Group by 'ticker', 'year', and 'section', then sum the 'word_count'
word_count_agg = mandatory_data.groupby(['ticker', 'year', 'section'], as_index=False)['word_count'].sum()

# 5. Pivot the DataFrame to create a multi-index with 'ticker' and 'year'
word_count_pivot = word_count_agg.pivot_table(
    index=['ticker', 'year'],
    columns='section',
    values='word_count',
    fill_value=0  # Fill missing values with 0
)

# Optional: Flatten the column index if needed (not necessary in multi-index)
# word_count_pivot.columns = word_count_pivot.columns.get_level_values(0)

# 6. Reset index to ensure 'ticker' and 'year' are part of the index
word_count_pivot = word_count_pivot.reset_index().set_index(['ticker', 'year'])


In [13]:
word_count_pivot

section      ACCOUNTING_FEES  BUSINESS  COMPENSATION  CONTROLS_AND_PROCEDURES  \
ticker year                                                                     
META   2013           38.000   107.000        38.000                  345.000   
       2014           34.000   107.000        34.000                  444.000   
       2015           34.000   173.000        34.000                  440.000   
       2016           34.000   173.000        34.000                  439.000   
       2017           34.000   173.000        34.000                  439.000   
       2018           34.000  2154.000     11633.000                  439.000   
       2019           34.000  1853.000     11576.000                  439.000   
       2020           34.000   212.000        34.000                  439.000   
       2021           38.000    88.000        38.000                  446.000   
       2022           38.000    33.000        38.000                  446.000   
       2023            0.000    66.000        38.000                  445.000   
       2024            0.000   160.000        38.000                  445.000   

section      EXHIBITS  FINANCIAL_STATEMENTS  MANAGEMENT  \
ticker year                                               
META   2013    13.000             10050.000   10422.000   
       2014    13.000              9009.000    8243.000   
       2015    13.000              9388.000    7237.000   
       2016    13.000              8453.000    7747.000   
       2017    13.000              9988.000    8545.000   
       2018  1261.000               272.000      32.000   
       2019  1241.000               422.000      25.000   
       2020   195.000             11312.000    7767.000   
       2021   190.000             12883.000    9594.000   
       2022     0.000             13240.000    9395.000   
       2023   218.000             15213.000   10489.000   
       2024   218.000             14933.000    9333.000   

section      MANAGEMENT_DISCUSSION  PRINCIPAL_STOCKHOLDERS  RISK_FACTORS  
ticker year                                                               
META   2013              10422.000                  38.000         0.000  
       2014               8243.000                  34.000         0.000  
       2015               7237.000                  34.000     14522.000  
       2016               7747.000                  34.000     15400.000  
       2017               8545.000                  34.000     16548.000  
       2018               8339.000                  34.000     17118.000  
       2019               7929.000                  34.000     18148.000  
       2020               7767.000                  34.000     21689.000  
       2021               9594.000                  38.000     25126.000  
       2022               9395.000                  38.000     26299.000  
       2023              10489.000                  38.000     26036.000  
       2024               9333.000                  38.000     27912.000

In [10]:
word_count_pivot

section      ACCOUNTING_FEES  BUSINESS  COMPENSATION  CONTROLS_AND_PROCEDURES  \
ticker year                                                                     
AAPL   2011           29.000  4839.000     13454.000                  646.000   
       2012           76.000  4763.000     13181.000                  644.000   
       2013            0.000   563.000         0.000                  652.000   
       2014            0.000   582.000         0.000                  653.000   
       2015            0.000   550.000         0.000                  653.000   
       2016           18.000   537.000        18.000                  635.000   
       2017           18.000   552.000        26.000                  635.000   
       2018           18.000   545.000        26.000                  635.000   
       2019           18.000   345.000        26.000                  635.000   
       2020            0.000   297.000        77.000                  642.000   
       2021            0.000   184.000        34.000                  642.000   
       2022            0.000   920.000        20.000                  642.000   
       2023            0.000   442.000        20.000                  636.000   

section      EXHIBITS  FINANCIAL_STATEMENTS  MANAGEMENT  \
ticker year                                               
AAPL   2011 15685.000                41.000    8740.000   
       2012  5485.000                57.000    9074.000   
       2013     0.000             13676.000    8916.000   
       2014     0.000             14159.000    9194.000   
       2015     0.000             13244.000    8300.000   
       2016     0.000             11799.000    8005.000   
       2017     0.000             11927.000    8091.000   
       2018     0.000             10412.000    7470.000   
       2019     0.000             10338.000    4183.000   
       2020     0.000             11290.000    4381.000   
       2021     0.000              9289.000    2553.000   
       2022     0.000              8192.000    2348.000   
       2023     0.000              6678.000    2123.000   

section      MANAGEMENT_DISCUSSION  PRINCIPAL_STOCKHOLDERS  RISK_FACTORS  
ticker year                                                               
AAPL   2011               8740.000                   0.000      7529.000  
       2012               9074.000                  51.000      7177.000  
       2013               8916.000                  51.000      7532.000  
       2014               9194.000                  51.000      7866.000  
       2015               8300.000                  51.000      7931.000  
       2016               8005.000                  18.000      7769.000  
       2017               8091.000                  18.000      7873.000  
       2018               7470.000                  18.000      8172.000  
       2019               4183.000                  18.000      8114.000  
       2020               4381.000                   0.000      9015.000  
       2021               2553.000                  34.000      9835.000  
       2022               2348.000                  20.000     10405.000  
       2023               2123.000                  20.000      9951.000

In [5]:
data["category"].value_counts()

category
NarrativeText    7607
ListItem           98
Name: count, dtype: int64

In [6]:
data["section"].value_counts()

section
FINANCIAL_STATEMENTS                   2679
RISK_FACTORS                           1293
MANAGEMENT_DISCUSSION                  1250
MANAGEMENT                             1250
COMPENSATION                            426
EXHIBITS                                161
BUSINESS                                151
MARKET_FOR_REGISTRANT_COMMON_EQUITY     138
MARKET_RISK_DISCLOSURES                 133
CONTROLS_AND_PROCEDURES                 116
LEGAL_PROCEEDINGS                        41
PROPERTIES                               30
PRINCIPAL_STOCKHOLDERS                   15
RELATED_PARTY_TRANSACTIONS               12
ACCOUNTING_FEES                          10
Name: count, dtype: int64

In [7]:
data["tag"].value_counts()

tag
p       2345
span    2089
font    1951
td      1223
tr        49
div       47
a          1
Name: count, dtype: int64

In [8]:
data

,cik,ticker,date,company,accession_number,section,order,category,tag,row_id,full_text
0,320193,AAPL,2011-10-26,APPLE INC,0001193125-11-282113,BUSINESS,0,NarrativeText,p,e19f7471d8fb867c5bc0026bc798ad53,Apple Inc. and its wholly-owned subsidiaries (...
1,320193,AAPL,2011-10-26,APPLE INC,0001193125-11-282113,BUSINESS,1,NarrativeText,p,1ce3d4c53620d2d099ab59da4ebc3b9e,The Company is committed to bringing the best ...
2,320193,AAPL,2011-10-26,APPLE INC,0001193125-11-282113,BUSINESS,2,NarrativeText,p,a8eda7d4be126f5088578bb5b355dc0d,The Company believes a high-quality buying exp...
3,320193,AAPL,2011-10-26,APPLE INC,0001193125-11-282113,BUSINESS,3,NarrativeText,p,834e8b74a0fbb0cdd5fd390f1979bfac,The Company’s retail stores are typically loca...
4,320193,AAPL,2011-10-26,APPLE INC,0001193125-11-282113,BUSINESS,4,NarrativeText,p,1e2460c44fe3255902521a729dabc0ae,"Throughout its history, the Company has been c..."
...,...,...,...,...,...,...,...,...,...,...,...
7700,320193,AAPL,2023-11-03,Apple Inc.,0000320193-23-000106,MANAGEMENT,54,NarrativeText,span,0cedd2f8c148b1fc82f4dc0f5c9f35b8,The Company is subject to income taxes in the ...
7701,320193,AAPL,2023-11-03,Apple Inc.,0000320193-23-000106,MANAGEMENT,55,NarrativeText,span,b771b34af4ca51d17d3fbf42777c02c0,The Company is subject to various legal procee...
7702,320193,AAPL,2023-11-03,Apple Inc.,0000320193-23-000106,COMPENSATION,0,NarrativeText,span,f8825199078db1af16a75e19d38c8eff,The information required by this Item will be ...
7703,320193,AAPL,2023-11-03,Apple Inc.,0000320193-23-000106,PRINCIPAL_STOCKHOLDERS,0,NarrativeText,span,f8825199078db1af16a75e19d38c8eff,The information required by this Item will be ...


In [4]:
sov.data("sec/10k", tickers=["META"], limit=1)

,cik,ticker,date,company,accession_number,section,order,category,tag,row_id,full_text
0,1326801,META,2013-02-01,Facebook Inc,0001326801-13-000003,BUSINESS,0,NarrativeText,font,a6145cfc83491272af346a0eae2c3336,We allocate the fair value of purchase conside...
1,1326801,META,2013-02-01,Facebook Inc,0001326801-13-000003,PROPERTIES,0,NarrativeText,font,e0e664ee42b78d089f4ac1248e533e86,"As of December 31, 2012, we leased office and ..."
2,1326801,META,2013-02-01,Facebook Inc,0001326801-13-000003,LEGAL_PROCEEDINGS,0,NarrativeText,font,8ae3ba693a1fbd26c64a636d5c0d7aca,Paul D. Ceglia filed suit against us and Mark ...
3,1326801,META,2013-02-01,Facebook Inc,0001326801-13-000003,LEGAL_PROCEEDINGS,1,NarrativeText,font,0dc121ed1c32cf81c4195c359707ec12,"Beginning on May 22, 2012, multiple putative c..."
4,1326801,META,2013-02-01,Facebook Inc,0001326801-13-000003,LEGAL_PROCEEDINGS,2,NarrativeText,font,322930695bbf6c305a377de61245609b,We are also party to various legal proceedings...
...,...,...,...,...,...,...,...,...,...,...,...
10545,1326801,META,2024-02-02,"Meta Platforms, Inc.",0001326801-24-000012,EXHIBITS,3,NarrativeText,td,efc44904ae01a497b5549538d5938573,"Certification of Susan Li, Chief Financial Off..."
10546,1326801,META,2024-02-02,"Meta Platforms, Inc.",0001326801-24-000012,EXHIBITS,4,NarrativeText,td,83e1d06a719c9d49143c8e856969b30c,"Certification of Mark Zuckerberg, Chief Execut..."
10547,1326801,META,2024-02-02,"Meta Platforms, Inc.",0001326801-24-000012,EXHIBITS,5,NarrativeText,td,7a2dae8e4a37cb40603f71c65bb2a9ed,"Certification of Susan Li, Chief Financial Off..."
10548,1326801,META,2024-02-02,"Meta Platforms, Inc.",0001326801-24-000012,EXHIBITS,6,NarrativeText,span,ed1d31dff6a985dfad8bc386b6c565fd,+ Indicates a management contract or compensat...
